<a href="https://colab.research.google.com/github/37stu37/FFE/blob/master/FFE_numba_Dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
%%time 
%%capture
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 

CPU times: user 676 ms, sys: 142 ms, total: 817 ms
Wall time: 24.1 s


In [65]:
# Load the Drive helper and mount
from google.colab import drive
%tensorflow_version 2.x
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
%%time
import datetime
import glob
from math import sqrt
import os
import matplotlib.pyplot as plt
import bokeh
import numpy as np
import pandas as pd
import geopandas as gpd
from scipy.spatial import distance
from shapely.geometry import box
from shapely.geometry import shape
from shapely.geometry import Point
import networkx as nx
from sys import getsizeof
from numba import jit
import dask.dataframe as dd
import dask.array as da
import dask
from dask.distributed import Client
from dask.diagnostics import ProgressBar
%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'

CPU times: user 6.23 ms, sys: 0 ns, total: 6.23 ms
Wall time: 6.44 ms


In [67]:
client = Client(processes=False)
client

/usr/local/lib/python3.6/dist-packages/distributed/bokeh/core.py:57: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 99] Cannot assign requested address
  warnings.warn('\n' + msg)
/usr/local/lib/python3.6/dist-packages/distributed/deploy/local.py:197: UserWarning: 
Could not launch service 'bokeh' on port 8787. Got the following message:

[Errno 99] Cannot assign requested address
  self.scheduler.start(scheduler_address)


Client Scheduler: inproc://172.28.0.2/124/8,Cluster Workers: 1 Cores: 2 Memory: 13.66 GB


Set up the path  to data and output

In [68]:
path = '/content/drive/My Drive/05_Sync/FFE/FireNetwork/00_input'
path_output = '/content/drive/My Drive/05_Sync/FFE/FireNetwork/00_output'

!ls "/content/drive/My Drive/05_Sync/FFE/FireNetwork/00_input"
!ls '/content/drive/My Drive/05_Sync/FFE/FireNetwork/00_output/dask_edge_list'

buildings_raw.cpg      buildings_raw_pts.mshp  buildings_raw.shp
buildings_raw.dbf      buildings_raw_pts.prj   buildings_raw.shx
buildings_raw.prj      buildings_raw_pts.shp   GD_wind.csv
buildings_raw_pts.cpg  buildings_raw_pts.shx   source_target.csv
buildings_raw_pts.dbf  buildings_raw.qpj
edge_data.parquet


Create the functions to be used by the algorithm

In [0]:
# wind scenario
def wind_scenario(file_name):
    # wind scenario conditions
    wind_data = pd.read_csv(os.path.join(path, file_name))
    i = np.random.randint(0, wind_data.shape[0])
    w = wind_data.iloc[i, 2]
    d = wind_data.iloc[i, 1]
    b = wind_data.iloc[i, 3]
    # wind direction
    wind_bearing_max = b + 45
    wind_bearing_min = b - 45
    if b == 360:
        wind_bearing_max = 45
    if b <= 0:  # should not be necessary
        wind_bearing_min = 0
    if b == 999:
        wind_bearing_max = 999
        wind_bearing_min = 0
    
    return wind_bearing_max, wind_bearing_min, d

In [0]:
# create ignitions
def add_random(df):
    random_column = np.random.random(0, 1, size=len(df.source))

    return random_column

def ignitions(edges):
    # add random column
    x = da.random.random(size=len(edges))
    edges['rng'] = x
    # fires_src = dd.concat(edges, dd.from_dask_array(x))
    # filter on random column
    fires_src = edges[edges.rng > edges.IgnProb_bl]

    return fires_src

In [0]:
# conditions of fire propagation
def conditions_spread(fires,burn,wind_bearing_max,wind_bearing_min,wind_distance):
    # add columns to ddf
    fires['wind_bearing_max'] = wind_bearing_max
    fires['wind_bearing_min'] = wind_bearing_min
    fires['wind_distance'] = wind_distance
    # wind speed -> neighbors selection from wind buffer
    fires = fires[fires.distance < fires.wind_distance]
    # wind direction
    fires = fires[(fires.bearing < fires.wind_bearing_max) & (fires.bearing < fires.wind_bearing_min)]
    # should not be already burnt
    burn_list = list(set(burn.source))
    fires = fires[~fires.source.isin(burn_list)]
    return fires

In [0]:
# burned assets
def burnt_assets(burn, fires):
  data = [burn, fires]
  burnt = dd.concat(data, axis=0)
  return burnt

In [0]:
# spreading fire
def new_fires(fires, edges):
  fires_list = list(set(fires.target))
  new_fires = edges[edges.source.isin(fires_list)]
  return new_fires

In [0]:
def main(number_of_scenarios, edges):
  list_BURNED = []
  # --- SCENARIOS
  print("number of scenarios : {}".format(number_of_scenarios))
  for scenario in range(number_of_scenarios):
      BURNED = pd.DataFrame(columns=['source','target','distance', 'bearing','IgnProb_bl'])
      BURNED = dd.from_pandas(BURNED, npartitions=1)
      print("--- SCENARIO : {}".format(scenario))
      FIRES = ignitions(EDGES)
      if len(FIRES.index) == 0:
          print("no fire")
          continue
      wind_bearing_max, wind_bearing_min, wind_distance = wind_scenario('GD_wind.csv') # no filtering, just adding wind info to dataframe
      # --------- STEPS
      for step in range(len(EDGES)):
          print("--------- STEP : {}".format(step))
          FIRES = conditions_spread(FIRES, BURNED, wind_bearing_max, wind_bearing_min, wind_distance)
          FIRES = new_fires(FIRES, edges)
          BURNED = burnt_assets(BURNED, FIRES)
  
  list_BURNED.append(BURNED)


In [0]:
# EDGES = dd.read_parquet(os.path.join(path_output, 'dask_edge_list', 'edge_data.parquet'), engine='pyarrow')
# print(EDGES)

In [0]:
%%time
# with ProgressBar():
# load data
# EDGES = dd.read_parquet(os.path.join(path_output, 'dask_edge_list', 'edge_data.parquet'), engine='pyarrow')
EDGES = pd.read_parquet(os.path.join(path_output, 'dask_edge_list', 'edge_data.parquet'), engine='pyarrow')
# run main
list_BURN = main(1, EDGES)

number of scenarios : 1
--- SCENARIO : 0
--------- STEP : 0
--------- STEP : 1
--------- STEP : 2
--------- STEP : 3
--------- STEP : 4
--------- STEP : 5
--------- STEP : 6
--------- STEP : 7
--------- STEP : 8
--------- STEP : 9
--------- STEP : 10
--------- STEP : 11
--------- STEP : 12
--------- STEP : 13
--------- STEP : 14
--------- STEP : 15
--------- STEP : 16
--------- STEP : 17
--------- STEP : 18
--------- STEP : 19
--------- STEP : 20
--------- STEP : 21
--------- STEP : 22


distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)


--------- STEP : 23
--------- STEP : 24


distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)


--------- STEP : 25


distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)


--------- STEP : 26
--------- STEP : 27


distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)


--------- STEP : 28
--------- STEP : 29
--------- STEP : 30
--------- STEP : 31
--------- STEP : 32
--------- STEP : 33
--------- STEP : 34
--------- STEP : 35
--------- STEP : 36
--------- STEP : 37
--------- STEP : 38
--------- STEP : 39
--------- STEP : 40
--------- STEP : 41
--------- STEP : 42
--------- STEP : 43
--------- STEP : 44
--------- STEP : 45
--------- STEP : 46


In [0]:
%%time
# check convergence
from itertools import groupby
number_of_burns_per_scenarios = [len(list(group)) for key, group in groupby(scos)]
cumulative_number_of_burns_per_scenarios = list(np.cumsum(number_of_burns_per_scenarios))
scenario_list = list(set(scos))
scenario_list = [x+1 for x in scenario_list]

average_burn_per_scenario = [c / s for c,s in zip(cumulative_number_of_burns_per_scenarios, scenario_list)]

In [0]:
plt.plot(scenario_list, average_burn_per_scenario)